In [54]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle
import cv2

In [39]:
# import pickle file from ./test_policy/test_trajectory.pkl
with open('./test_policy/test_fully-connected_10/test_trajectory.pkl', 'rb') as f:
    trajectories = pickle.load(f)

episode = 20
trajectory = trajectories[episode]

# print(len(trajectory[-2]) ,trajectory[-2])
# calculate cumulative reward for each agent until each timestep as additional columns
for trajectory in trajectories:
    for i in range(len(trajectory)):
        trajectory[i].extend([sum([row[14] for row in trajectory[:i+1]]), sum([row[15] for row in trajectory[:i+1]]), sum([row[16] for row in trajectory[:i+1]])])

# print(len(trajectory[-2]) ,trajectory[-2])


for trajectory in trajectories:
    for k in range(len(trajectory)):
        dists = []
        for i in range(3):
            dist = []
            for j in range(3):
                # print(np.array(row[2+2*i:4+2*i]))
                dist.append(np.linalg.norm(np.array(trajectory[k][2+2*i:4+2*i]) - np.array(trajectory[k][8+2*j:10+2*j])))
            dists.append(min(dist))
        # print(len(dists))
        trajectory[k].extend(dists)


# trajectory = [row.extend(distss[i]) for i, row in enumerate(trajectory)]

print(len(trajectory[0]))

# trajectories is a list of trajectories. Where each trajectory is a list of:
# [timestep, agent1_x, agent1_y, agent2_x, agent2_y, agent3_x, agent3_y, landmark1_x, landmark1_y, landmark2_x, landmark2_y, landmark3_x, landmark3_y, reward_1, reward_2, reward_3]
# convert trajectories to pd dataframe with columns: episode, timestep, agent1_x, agent1_y, agent2_x, agent2_y, agent3_x, agent3_y, reward_1, reward_2, reward_3
# Flatten the nested list
flattened_data = [tup for trajectory in trajectories for tup in trajectory]
# Convert to DataFrame
df = pd.DataFrame(flattened_data, columns=[ 'episode', 'timestep',
                                            'agent1_x', 'agent1_y',
                                            'agent2_x', 'agent2_y', 
                                            'agent3_x', 'agent3_y', 
                                            'land1_x', 'land1_y',
                                            'land2_x', 'land2_y', 
                                            'land3_x', 'land3_y', 
                                            'reward_1', 'reward_2', 'reward_3',
                                            'agent1_cum_reward', 'agent2_cum_reward', 'agent3_cum_reward',
                                            'agent1_dist', 'agent2_dist', 'agent3_dist'])



BLUE = [0, 0.4470, 0.7410]
RED = [0.8500, 0.3250, 0.0980]
YELLOW = [0.929, 0.6940, 0.1250]


df.head()




23


,episode,timestep,agent1_x,agent1_y,agent2_x,agent2_y,agent3_x,agent3_y,land1_x,land1_y,...,land3_y,reward_1,reward_2,reward_3,agent1_cum_reward,agent2_cum_reward,agent3_cum_reward,agent1_dist,agent2_dist,agent3_dist
0,1,1,-0.569010,0.215131,-0.650407,-0.312304,0.436089,-0.543599,0.5,0.0,...,-0.433013,0.001955,0.001955,0.001955,0.001955,0.001955,0.001955,0.386315,0.418206,0.547344
1,1,2,-0.483568,0.215382,-0.564262,-0.312430,0.434339,-0.543597,0.5,0.0,...,-0.433013,0.005798,0.005798,0.005798,0.007753,0.007753,0.007753,0.319244,0.336602,0.547548
2,1,3,-0.369608,0.215556,-0.460854,-0.313397,0.432026,-0.543594,0.5,0.0,...,-0.433013,0.014924,0.014924,0.014924,0.022678,0.022678,0.022678,0.248180,0.242420,0.547827
3,1,4,-0.234602,0.215751,-0.335352,-0.312720,0.429292,-0.543592,0.5,0.0,...,-0.433013,0.024801,0.024801,0.024801,0.047479,0.047479,0.047479,0.217807,0.147497,0.548172
4,1,5,-0.085972,0.217183,-0.192920,-0.310577,0.426246,-0.543696,0.5,0.0,...,-0.433013,0.019687,0.019687,0.019687,0.067165,0.067165,0.067165,0.271086,0.135088,0.548676


## Episodic Analysis

In [43]:
%matplotlib qt

episode = 923


plt.figure(figsize=(15,4.5))

plt.subplot(1,3,1)
# plot the x,y trajectory of agents in episode 1 sorted by timestep using seaborn
sns.lineplot(x="agent1_x", y="agent1_y", data=df[df['episode']==episode], sort=False, color=BLUE)
sns.lineplot(x="agent2_x", y="agent2_y", data=df[df['episode']==episode], sort=False, color=RED)
sns.lineplot(x="agent3_x", y="agent3_y", data=df[df['episode']==episode], sort=False, color=YELLOW)
# mark the location of landmarks
sns.scatterplot(x="land1_x", y="land1_y", data=df[df['episode']==episode], color='gray', s=300)
sns.scatterplot(x="land2_x", y="land2_y", data=df[df['episode']==episode], color='gray', s=300)
sns.scatterplot(x="land3_x", y="land3_y", data=df[df['episode']==episode], color='gray', s=300)
# make axis equal and limit the axis to -1.0 to 1.0
plt.axis('square')
plt.xlim(-1.5,1.5)
plt.ylim(-1.5,1.5)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

plt.subplot(1,3,2)
# plot the reward of agents in episode 1 sorted by timestep using seaborn
sns.lineplot(x="timestep", y="reward_1", data=df[df['episode']==episode], sort=False, color=BLUE, label='agent1')
sns.lineplot(x="timestep", y="reward_2", data=df[df['episode']==episode], sort=False, color=RED, label='agent2')
sns.lineplot(x="timestep", y="reward_3", data=df[df['episode']==episode], sort=False, color=YELLOW, label='agent3')
plt.xlabel('timestep')
plt.ylabel('shared reward')
plt.title('Episode {}'.format(episode))

plt.subplot(1,3,3)
# plot the distance to the closest landmark for each agent
sns.lineplot(x="timestep", y="agent1_dist", data=df[df['episode']==episode], sort=False, color=BLUE, label='agent1')
sns.lineplot(x="timestep", y="agent2_dist", data=df[df['episode']==episode], sort=False, color=RED, label='agent2')
sns.lineplot(x="timestep", y="agent3_dist", data=df[df['episode']==episode], sort=False, color=YELLOW, label='agent3')
plt.xlabel('timestep')
plt.ylabel('distance to closest landmark')
plt.show()



## Behavior Reply Animation

In [57]:

%matplotlib qt
import pygame
import time

episode = 923
# Sample trajectory data
trajectory_ = trajectories[episode]
# drop the first element of the list for each list item in trajectory
trajectory_ = [t[1:] for t in trajectory_]


pygame.init()

# Display settings
WIDTH, HEIGHT = 640, 480
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Multi-Agent and Landmarks Animation")

# Colors
BLUE = np.array([0, 0.4470*255, 0.7410*255])
RED = np.array([0.8500*255, 0.3250*255, 0.0980*255])
YELLOW = np.array([0.929*255, 0.6940*255, 0.1250*255])

WHITE = (255, 255, 255)
AGENTS_COLORS = [BLUE, RED, YELLOW]
LANDMARK_COLORS = [(50, 50, 50), (50, 50, 50), (50, 50, 50)]
BLACK = (20, 20, 20)

frames = []



# shift and scale the data (-1,1) to fit the screen size (0,500) and (0,500)

SCALE_FACTOR = 200  # given our dimensions and trajectory range
SCREEN_CENTER = (WIDTH // 2, HEIGHT // 2)

# Choose a font (using a default system font here)
font = pygame.font.SysFont("arial", 16)

def map_to_screen(pos):
    """Map a trajectory position to a screen position."""
    return int(pos[0] * SCALE_FACTOR + SCREEN_CENTER[0]), int(pos[1] * SCALE_FACTOR + SCREEN_CENTER[1])


def draw_entity(screen, x, y, color, size=.2*SCALE_FACTOR):
    pygame.draw.circle(screen, color, (int(x), int(y)), size)



def display_text(text, x, y, color=BLACK):
    """Render and display text on the screen at specified coordinates."""
    text_surface = font.render(text, True, color)
    screen.blit(text_surface, (x, y))

def capture_frame(screen):
    """Capture the current Pygame screen frame."""
    frame = pygame.Surface(screen.get_size())
    frame.blit(screen, (0, 0))
    frames.append(frame)


def save_frames_to_video(frames, filename, fps=30):
    """Save captured frames to a video file."""
    height, width = frames[0].get_size()
    size = (width, height)
    out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'XVID'), fps, size)
    for frame in frames:
        # Convert Pygame surface to OpenCV format
        frame_rgb = pygame.surfarray.array3d(frame)  
        frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
        out.write(frame_bgr)

    out.release()

def main():
    time.sleep(5)
    clock = pygame.time.Clock()
    running = True
    current_time = 0

    while running:
        screen.fill(WHITE)

        
        # Calculate the current positions of the agents and landmarks
        for i in range(len(trajectory_) - 1):

            cum_rew_1 = trajectory_[i][16]
            cum_rew_2 = trajectory_[i][17]
            cum_rew_3 = trajectory_[i][18]

            

            t0, *data0 = trajectory_[i]
            t1, *data1 = trajectory_[i + 1]

            if t0 <= current_time < t1:
                alpha = (current_time - t0) / (t1 - t0)

                # Drawing agents
                for j in range(3):  
                    x0, y0, x1, y1 = data0[j * 2], data0[j * 2 + 1], data1[j * 2], data1[j * 2 + 1]
                    # r0, r1 = data0[12 + j], data1[12 + j]
                    x = x0 * (1 - alpha) + x1 * alpha
                    y = y0 * (1 - alpha) + y1 * alpha
                    # r = r0 * (1 - alpha) + r1 * alpha
                    r = data0[12 + j]
                    screen_x, screen_y = map_to_screen((x, y))
                    draw_entity(screen, screen_x, screen_y, AGENTS_COLORS[j])
                    display_text(f"{r:.4f}", screen_x-25, screen_y-60)

                # Drawing landmarks
                for j in range(3):
                    x0, y0, x1, y1 = data0[6 + j * 2], data0[7 + j * 2], data1[6 + j * 2], data1[7 + j * 2]
                    x = x0 * (1 - alpha) + x1 * alpha
                    y = y0 * (1 - alpha) + y1 * alpha
                    screen_x, screen_y = map_to_screen((x, y))
                    draw_entity(screen, screen_x, screen_y, LANDMARK_COLORS[j], size=.05*SCALE_FACTOR)

                display_text(f"agent 1 cumulative reward: {cum_rew_1:.2f}", 370, 10)
                display_text(f"agent 2 cumulative reward: {cum_rew_2:.2f}", 370, 30)
                display_text(f"agent 2 cumulative reward: {cum_rew_3:.2f}", 370, 50)
                display_text(f"episode: {episode}",10, 20)
                display_text(f"timestep: {i}", 120, 20)

        pygame.display.flip()   
        capture_frame(screen)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        
        clock.tick(60)
        # Update current time
        current_time += .1

        # Exit loop when trajectory ends
        if current_time > trajectory[-1][0]:
            running = False

    pygame.quit()

if __name__ == "__main__":
    main()
    save_frames_to_video(frames,'episode_{}.avi'.format(episode))



KeyboardInterrupt: 

## Mean Performance Analysis

In [34]:
# Colors
BLUE = [0, 0.4470, 0.7410]
RED = [0.8500, 0.3250, 0.0980]
YELLOW = [0.929, 0.6940, 0.1250]



fig = plt.figure(figsize=(15,8))
plt.subplot(2,2,1)
sns.lineplot(x="timestep", y="agent1_dist", data=df, errorbar=("sd",1), sort=False, color=BLUE, label='agent1')
sns.lineplot(x="timestep", y="agent2_dist", data=df, errorbar=("sd",1), sort=False, color=RED, label='agent2')
sns.lineplot(x="timestep", y="agent3_dist", data=df, errorbar=("sd",1), sort=False, color=YELLOW, label='agent3')
plt.xlabel('timestep')
plt.ylabel('distance to closest landmark')
plt.grid()

plt.subplot(2,2,2)
# sns.violinplot(data=df, x="timestep", y="reward_1")
sns.histplot(data=df[df['timestep']==70], x="agent1_cum_reward", bins=70)
plt.xlabel('agent 1 cum. shared reward')
plt.ylabel('number of episodes')
plt.grid()

plt.subplot(2,2,3)
sns.lineplot(x="timestep", y="agent1_cum_reward", data=df, errorbar=("sd",1), sort=False, color=BLUE, label='agent1')
sns.lineplot(x="timestep", y="agent2_cum_reward", data=df, errorbar=("sd",1), sort=False, color=RED, label='agent2')
sns.lineplot(x="timestep", y="agent3_cum_reward", data=df, errorbar=("sd",1), sort=False, color=YELLOW, label='agent3')
plt.xlabel('timestep')
plt.ylabel('mean cumulative shared reward')
plt.grid()

plt.subplot(2,2,4)
sns.lineplot(x="timestep", y="reward_1", data=df, errorbar=("sd",1), sort=False, color=BLUE, label='agent1')
sns.lineplot(x="timestep", y="reward_2", data=df, errorbar=("sd",1), sort=False, color=RED, label='agent2')
sns.lineplot(x="timestep", y="reward_3", data=df, errorbar=("sd",1), sort=False, color=YELLOW, label='agent3')
plt.xlabel('timestep')
plt.ylabel('mean shared reward')
plt.grid()
plt.show()

fig.tight_layout()




<Axes: xlabel='agent1_cum_reward', ylabel='Count'>

In [122]:
# find the episodes with the lost cumulative reward at the timestep 70
df[df['timestep']==70].sort_values(by='agent1_cum_reward', ascending=True).head(10)


,episode,timestep,agent1_x,agent1_y,agent2_x,agent2_y,agent3_x,agent3_y,land1_x,land1_y,...,land3_y,reward_1,reward_2,reward_3,agent1_cum_reward,agent2_cum_reward,agent3_cum_reward,agent1_dist,agent2_dist,agent3_dist
17359,248,70,-0.543376,-1.027080,-2.965943,2.324070,1.973199,1.491431,0.5,0.0,...,-0.433013,2.765631e-69,2.765631e-69,2.765631e-69,4.652545e-13,4.652545e-13,4.652545e-13,0.662560,3.309448,2.096349
54109,773,70,0.521156,1.194668,2.440394,2.279588,2.603815,2.645601,0.5,0.0,...,-0.433013,2.284251e-94,2.284251e-94,2.284251e-94,4.739723e-10,4.739723e-10,4.739723e-10,1.083882,2.993602,3.380125
21909,313,70,0.326593,0.294468,1.628497,2.385157,1.166466,2.432902,0.5,0.0,...,-0.433013,1.484994e-57,1.484994e-57,1.484994e-57,1.280297e-09,1.280297e-09,1.280297e-09,0.341733,2.638651,2.450701
31009,443,70,0.322937,1.009739,1.187097,1.181089,0.841576,1.090427,0.5,0.0,...,-0.433013,2.242209e-17,2.242209e-17,2.242209e-17,3.335787e-08,3.335787e-08,3.335787e-08,0.812939,1.366409,1.142675
32549,465,70,0.698887,0.957473,0.645835,-1.117532,0.273729,-1.017848,0.5,0.0,...,-0.433013,4.507780e-13,4.507780e-13,4.507780e-13,8.347150e-08,8.347150e-08,8.347150e-08,0.977911,1.127008,0.785063
40529,579,70,0.650833,-8.734472,-1.003702,1.211598,-1.096818,0.755905,0.5,0.0,...,-0.433013,3.297303e-312,3.297303e-312,3.297303e-312,1.231898e-07,1.231898e-07,1.231898e-07,8.350193,1.083634,0.906289
32059,458,70,0.552698,-2.500751,-0.547425,-2.889860,-0.302394,-2.563204,0.5,0.0,...,-0.433013,2.068707e-68,2.068707e-68,2.068707e-68,1.663775e-07,1.663775e-07,1.663775e-07,2.218077,2.474785,2.130835
19879,284,70,0.526848,-12.756290,0.383371,1.161758,-0.011709,0.929286,0.5,0.0,...,-0.433013,0.000000e+00,0.000000e+00,0.000000e+00,1.476803e-06,1.476803e-06,1.476803e-06,12.347739,0.965520,0.550518
5599,80,70,0.445426,-0.482668,-0.291745,-1.404208,-0.019340,-1.138117,0.5,0.0,...,-0.433013,3.027747e-08,3.027747e-08,3.027747e-08,7.296082e-06,7.296082e-06,7.296082e-06,0.485744,0.972092,0.741873
29679,424,70,0.559180,0.025038,-0.964309,2.268821,-1.157550,1.924041,0.5,0.0,...,-0.433013,7.896722e-31,7.896722e-31,7.896722e-31,1.005137e-05,1.005137e-05,1.005137e-05,0.064258,1.969880,1.745512


In [128]:
# print location of landmarks at timestep 70 episode 923
df[(df['timestep']==1) & (df['episode']==1)][['land1_x', 'land1_y', 'land2_x', 'land2_y', 'land3_x', 'land3_y']]


,land1_x,land1_y,land2_x,land2_y,land3_x,land3_y
0,0.5,0.0,-0.25,0.433013,-0.25,-0.433013


In [38]:
# plot a contour plot of the reward function as np.exp(-(d**2)/0.1) at the location of landmarks
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt


print(df[(df['timestep']==1) & (df['episode']==1)][['land1_x', 'land1_y', 'land2_x', 'land2_y', 'land3_x', 'land3_y']])


def reward_function(d):
    return np.exp(-(d**2)/.1)

x = np.linspace(-2, 2, 100)
y = np.linspace(-2, 2, 100)

X, Y = np.meshgrid(x, y)
lanmark_locations = [(0.5, 0.0), (-0.25, 0.433013), (-0.25, -0.433013)]

# calculate the distance to the closest landmark for each point in the grid
Z = np.zeros_like(X)
for i in range(len(X)):
    for j in range(len(Y)):
        dists = []
        for k in range(3):
            dists.append(np.linalg.norm(np.array([X[i,j], Y[i,j]]) - np.array(lanmark_locations[k])))
        Z[i,j] = reward_function(min(dists))



fig = plt.figure(figsize=(6,6))
ax = plt.axes(projection='3d')
ax.contour3D(X, Y, Z, 50, cmap='binary')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('individual reward')
ax.view_init(60, 35)
plt.show()

   land1_x  land1_y  land2_x   land2_y  land3_x   land3_y
0      0.5      0.0    -0.25  0.433013    -0.25 -0.433013
